In [90]:
import pandas as pd

In [91]:
# loading dataset 
train = pd.read_csv("./widsdatathon2020_data/training_v2.csv")
test = pd.read_csv("./widsdatathon2020_data/unlabeled.csv")

In [92]:
train.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.95,0,Caucasian,F,172.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.64,1,Caucasian,F,165.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,NaN,0,Caucasian,M,188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


In [93]:
train.columns

Index(['encounter_id', 'patient_id', 'hospital_id', 'hospital_death', 'age',
       'bmi', 'elective_surgery', 'ethnicity', 'gender', 'height',
       ...
       'aids', 'cirrhosis', 'diabetes_mellitus', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis', 'apache_3j_bodysystem',
       'apache_2_bodysystem'],
      dtype='object', length=186)

In [94]:
test.head(2)

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,2,127112,7,NaN,56.0,21.102439,0,Caucasian,M,180.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic
1,5,130737,178,NaN,NaN,19.500595,1,Caucasian,F,145.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic


In [95]:
train = train.select_dtypes(exclude=['object'])

In [96]:
dictionary = pd.read_csv('./widsdatathon2020_data/WiDS Datathon 2020 Dictionary.csv')

In [97]:
dictionary

,Category,Variable Name,Unit of Measure,Data Type,Description,Example
0,identifier,encounter_id,None,integer,Unique identifier associated with a patient un...,None
1,identifier,hospital_id,None,integer,Unique identifier associated with a hospital,None
2,identifier,patient_id,None,integer,Unique identifier associated with a patient,None
3,demographic,hospital_death,None,binary,Whether the patient died during this hospitali...,0
4,demographic,age,Years,numeric,The age of the patient on unit admission,None
5,demographic,bmi,kilograms/metres^2,string,The body mass index of the person on unit admi...,21.5
6,demographic,elective_surgery,None,binary,Whether the patient was admitted to the hospit...,0
7,demographic,ethnicity,None,string,The common national or cultural tradition whic...,Caucasian
8,demographic,gender,None,string,The genotypical sex of the patient,F
9,demographic,height,centimetres,numeric,The height of the person on unit admission,180


In [98]:
y = train['hospital_death']
X = train
X = train.drop('hospital_death',axis = 1)
test = test.drop('hospital_death',axis = 1)

In [99]:
# unique labels in the output
y.unique()

array([0, 1])

In [100]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [101]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#train a model
model = Sequential()
n_cols = x_train.shape[1]
model.add(Dense(1000,activation='relu', input_shape=(n_cols,)))
model.add(Dense(15,activation='relu'))
model.add(Dense(1))



In [102]:
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
sgd = optimizers.SGD(lr=0.01)


In [113]:
optimizer = model.compile(sgd, loss='binary_crossentropy',metrics=['accuracy'])

In [114]:
predictions = model.fit(x_train,y_train, epochs=10)

Epoch 1/10
73370/73370 [==============================] - 11s 155us/sample - loss: nan - acc: 0.9139
Epoch 2/10
73370/73370 [==============================] - 10s 139us/sample - loss: nan - acc: 0.9139
Epoch 3/10
73370/73370 [==============================] - 11s 148us/sample - loss: nan - acc: 0.9139
Epoch 4/10
73370/73370 [==============================] - 11s 152us/sample - loss: nan - acc: 0.9139
Epoch 5/10
73370/73370 [==============================] - 11s 144us/sample - loss: nan - acc: 0.9139
Epoch 6/10
73370/73370 [==============================] - 13s 174us/sample - loss: nan - acc: 0.9139
Epoch 7/10
73370/73370 [==============================] - 12s 165us/sample - loss: nan - acc: 0.9139
Epoch 8/10
73370/73370 [==============================] - 10s 142us/sample - loss: nan - acc: 0.9139
Epoch 9/10
73370/73370 [==============================] - 11s 150us/sample - loss: nan - acc: 0.9139
Epoch 10/10
73370/73370 [==============================] - 10s 141us/sample - loss: nan - a

In [ ]:
train.info()


In [47]:
model = Sequential()
model.add(Dense(128, input_shape=(131,), activation='relu', name='fc1'))
model.add(Dense(64, activation='relu', name='fc2'))
model.add(Dense(2, activation='softmax', name='output'))
from tensorflow.keras.optimizers import SGD, Adam

In [48]:
optimizer = SGD(lr=0.1)

In [49]:
model.compile(optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])